##### Imports

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import certifi
import json
from fredapi import Fred
import os

# Custom packages
import derive_data as dd

import warnings
warnings.filterwarnings("ignore")

# Environment variables
import dotenv
dotenv.load_dotenv()
FRED_API_KEY = os.getenv("FRED_API_KEY")
FMP_API_KEY = os.getenv("FMP_API_KEY")

# Data Collection

### Inputs

In [4]:
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

idx = pd.read_csv('data/inputs/index_symbols.csv')
comm = pd.read_csv('data/inputs/commodity_symbols.csv')

url = f"https://financialmodelingprep.com/stable/index-list?apikey={FMP_API_KEY}"
fmp_idx = pd.DataFrame(get_jsonparsed_data(url))
fmp_idx = fmp_idx[fmp_idx['symbol'].isin(idx['FMP API Symbol'])].reset_index(drop=True)
fmp_idx['fx_symbol'] = fmp_idx['currency'].apply(lambda x: x+'USD' if x != 'USD' else None)

url = f"https://financialmodelingprep.com/stable/commodities-list?apikey={FMP_API_KEY}"
fmp_comm = pd.DataFrame(get_jsonparsed_data(url))
fmp_comm = fmp_comm[fmp_comm['symbol'].isin(comm['FMP API Symbol'])].reset_index(drop=True)

fmp_idx.to_csv('data/inputs/fmp_index_list.csv', index=False)
fmp_comm.to_csv('data/inputs/fmp_commodity_list.csv', index=False)

date_from = '1990-01-01'
date_to = '2025-09-26'

### Equity Index, Commodity, and FX Daily Timeseries data 

In [16]:
# symbol = fmp_idx.loc[0, 'symbol']
# url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
# df = pd.DataFrame(get_jsonparsed_data(url))\
#     [['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'vwap']]
# df['date'] = pd.to_datetime(df['date'])
# df = df.set_index('date')
# df.columns = pd.MultiIndex.from_product([[df['symbol'].iloc[0]], df.columns])
# df = df.drop(columns=df.columns[0]).sort_index()

# for i in range(1, len(fmp_idx)):
#     symbol = fmp_idx.loc[i, 'symbol']
#     url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
#     temp = pd.DataFrame(get_jsonparsed_data(url))\
#         [['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'vwap']]
#     temp['date'] = pd.to_datetime(temp['date'])
#     temp = temp.set_index('date')
#     temp.columns = pd.MultiIndex.from_product([[temp['symbol'].iloc[0]], temp.columns])
#     temp = temp.drop(columns=temp.columns[0]).sort_index()
#     df = pd.concat([df, temp], axis=1)

# msci = pd.read_excel('data/inputs/MSCI_China_Index.xlsx')[:-1]
# msci['Date'] = pd.to_datetime(msci['Date'])
# # msci = msci.reindex(index=df.index)
# msci = msci.set_index('Date')
# msci.columns = pd.MultiIndex.from_product([['MSCI_China'], ['close']])
# msci[('MSCI_China', 'volume')] = np.nan

# df.join(msci, how='left').to_csv('data/processed/index_data.csv')

### -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------###

# symbol = fmp_comm.loc[0, 'symbol']
# url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
# df = pd.DataFrame(get_jsonparsed_data(url))\
#     [['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'vwap']]
# df['date'] = pd.to_datetime(df['date'])
# df = df.set_index('date')
# df.columns = pd.MultiIndex.from_product([[df['symbol'].iloc[0]], df.columns])
# df = df.drop(columns=df.columns[0]).sort_index()

# for i in range(1, len(fmp_comm)):
#     symbol = fmp_comm.loc[i, 'symbol']
#     url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
#     temp = pd.DataFrame(get_jsonparsed_data(url))\
#         [['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'vwap']]
#     temp['date'] = pd.to_datetime(temp['date'])
#     temp = temp.set_index('date')
#     temp.columns = pd.MultiIndex.from_product([[temp['symbol'].iloc[0]], temp.columns])
#     temp = temp.drop(columns=temp.columns[0]).sort_index()
#     df = pd.concat([df, temp], axis=1)

# # df.to_csv('data/commodity_data.csv')
# nickel = pd.read_csv('data/inputs/Nickel_futures.csv', parse_dates=['Date'], dayfirst=True, index_col='Date')\
#     .rename_axis('date').sort_index().rename(columns={'Price': 'close', 'Vol.': 'volume', 'Open': 'open', 'High': 'high', 'Low': 'low'})\
#         [['open', 'high', 'low', 'close', 'volume']]
# nickel.index.name = 'date'
# for col in nickel.columns:
#     if nickel[col].dtype == 'object':
#         nickel[col] = pd.to_numeric(nickel[col].astype(str).str.replace(',', ''), errors='coerce')
# nickel.columns = pd.MultiIndex.from_product([['Nickel'], nickel.columns])
# nickel[('Nickel', 'volume')] = np.nan
# df.join(nickel, how='left').to_csv('data/processed/commodity_data.csv')

### -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------###

# fx_symbols_retieved = []
# symbol = fmp_idx.loc[0, 'fx_symbol']
# fx_symbols_retieved.append(symbol)
# url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
# df = pd.DataFrame(get_jsonparsed_data(url))[['symbol', 'date', 'close']]
# df['date'] = pd.to_datetime(df['date'])
# df = df.set_index('date')
# df.rename(columns={'close': df['symbol'].iloc[0]}, inplace=True)
# df = df.drop(columns=df.columns[0]).sort_index()

# for i in range(1, len(fmp_idx)):
#     symbol = fmp_idx.loc[i, 'fx_symbol']
#     if symbol is None or symbol in fx_symbols_retieved:
#         continue
#     fx_symbols_retieved.append(symbol)
#     url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
#     temp = pd.DataFrame(get_jsonparsed_data(url))[['symbol', 'date', 'close']]
#     temp['date'] = pd.to_datetime(temp['date'])
#     temp = temp.set_index('date')
#     temp.rename(columns={'close': temp['symbol'].iloc[0]}, inplace=True)
#     temp = temp.drop(columns=temp.columns[0]).sort_index()
#     df = pd.concat([df, temp], axis=1)

# df.to_csv('data/processed/fx_data.csv')

### Macroeconomic Data

In [7]:
# Core macro and rate series
macro_codes_dict = {
    # RATE BENCHMARKS
    'FEDFUNDS': 'Federal Funds Effective Rate',
    'SOFR': 'Secured Overnight Financing Rate',
    # TREASURY RATES
    'GS1M': '1-Month Treasury Rate',
    'GS3M': '3-Month Treasury Rate', 
    'GS6M': '6-Month Treasury Rate',
    'GS1': '1-Year Treasury Rate',
    'GS2': '2-Year Treasury Rate',
    'GS3': '3-Year Treasury Rate',
    'GS5': '5-Year Treasury Rate',
    'GS7': '7-Year Treasury Rate',
    'GS10': '10-Year Treasury Rate',
    'GS20': '20-Year Treasury Rate',
    'GS30': '30-Year Treasury Rate',
    # OTHER RATES
    'AAA': 'Moody\'s Seasoned AAA Corporate Bond Yield',
    'BAA': 'Moody\'s Seasoned BAA Corporate Bond Yield',
    'MORTGAGE30US': '30-Year Fixed Rate Mortgage Average',
    # ECONOMIC INDICATORS
    'GDP': 'Gross Domestic Product',
    'GDPC1': 'Real Gross Domestic Product',
    'CPIAUCSL': 'Consumer Price Index',
    'CPILFESL': 'Core CPI (Less Food and Energy)',
    'UNRATE': 'Unemployment Rate',
    'CIVPART': 'Labor Force Participation Rate',
    'INDPRO': 'Industrial Production Index',
    'PAYEMS': 'Total Nonfarm Payrolls',
    'HOUST': 'Housing Starts',
    'PERMIT': 'Building Permits',
    # MONEY SUPPLY
    'M1SL': 'M1 Money Stock',
    'M2SL': 'M2 Money Stock',
    'BASE': 'St. Louis Adjusted Monetary Base',
    # MARKET INDICATORS
    'VIXCLS': 'CBOE Volatility Index (VIX)',
    'UMCSENT': 'University of Michigan Consumer Sentiment',
    'USSLIND': 'Leading Index for the United States',
    # ADDITIONAL RATES
    'DPRIME': 'Bank Prime Loan Rate'
}

macro_units = {
    # INTEREST RATES AND FINANCIAL RATES
    'FEDFUNDS': 'Percent, Seasonally Adjusted',
    'SOFR': 'Percent, Not Seasonally Adjusted',
    'GS1M': 'Percent, Not Seasonally Adjusted',
    'GS3M': 'Percent, Not Seasonally Adjusted',
    'GS6M': 'Percent, Not Seasonally Adjusted',
    'GS1': 'Percent, Not Seasonally Adjusted',
    'GS2': 'Percent, Not Seasonally Adjusted',
    'GS3': 'Percent, Not Seasonally Adjusted',
    'GS5': 'Percent, Not Seasonally Adjusted',
    'GS7': 'Percent, Not Seasonally Adjusted',
    'GS10': 'Percent, Not Seasonally Adjusted',
    'GS20': 'Percent, Not Seasonally Adjusted',
    'GS30': 'Percent, Not Seasonally Adjusted',
    'AAA': 'Percent, Not Seasonally Adjusted',
    'BAA': 'Percent, Not Seasonally Adjusted',
    'MORTGAGE30US': 'Percent, Not Seasonally Adjusted',
    'DPRIME': 'Percent, Not Seasonally Adjusted',
    # ECONOMIC INDICATORS
    'GDP': 'Billions of Dollars, Seasonally Adjusted Annual Rate',
    'GDPC1': 'Billions of Chained 2017 Dollars, Seasonally Adjusted Annual Rate',
    'CPIAUCSL': 'Index 1982-1984=100, Seasonally Adjusted',
    'CPILFESL': 'Index 1982-1984=100, Seasonally Adjusted',
    'UNRATE': 'Percent, Seasonally Adjusted',
    'CIVPART': 'Percent, Seasonally Adjusted',
    'INDPRO': 'Index 2017=100, Seasonally Adjusted',
    'PAYEMS': 'Thousands of Persons, Seasonally Adjusted',
    'HOUST': 'Thousands of Units, Seasonally Adjusted Annual Rate',
    'PERMIT': 'Thousands of Units, Seasonally Adjusted Annual Rate',
    # MONEY SUPPLY
    'M1SL': 'Billions of Dollars, Seasonally Adjusted',
    'M2SL': 'Billions of Dollars, Seasonally Adjusted',
    'BASE': 'Millions of Dollars, Not Seasonally Adjusted',
    # MARKET INDICATORS
    'VIXCLS': 'Index, Not Seasonally Adjusted',
    'UMCSENT': 'Index 1966:Q1=100, Not Seasonally Adjusted',
    'USSLIND': 'Index 2016=100, Not Seasonally Adjusted'
}


In [8]:
# def get_comprehensive_macro_data(fred_api_key, series_dict=macro_codes_dict, start_date='1990-01-01', end_date='2025-09-26'):
#     fred = Fred(api_key=fred_api_key)
#     all_data = pd.DataFrame(index=pd.DatetimeIndex(pd.date_range(start=start_date, end=end_date, freq='D')))
#     successful_series = []
#     failed_series = []
    
#     for code, description in series_dict.items():
#         try:
#             if code in ['MORTGAGE30US', 'BASE']:
#                 series_data = fred.get_series(code, obervation_satrt=start_date, frequency='m', aggregation_method='eop').rename(code)
#             else:
#                 series_data = fred.get_series(code, observation_start=start_date).rename(code)
#             if not series_data.empty:
#                 all_data = all_data.join(series_data, how='left')
#                 successful_series.append((code, description))
#                 # print(f"✓ {code}: {len(series_data)} observations")
#             else:
#                 failed_series.append((code, "No data in time range"))
#                 # print(f"✗ {code}: No data in specified time range")
#         except Exception as e:
#             failed_series.append((code, str(e)))
#             # print(f"✗ {code}: {e}")

#     all_data = all_data.sort_index()
#     all_data.index.name = 'date'
#     # print(f"Final dataset: {all_data.shape[0]} observations, {all_data.shape[1]} variables")
#     # print(f"Date range: {all_data.index.min()} to {all_data.index.max()}")
#     return all_data, successful_series, failed_series

# macro_data, successful, failed = get_comprehensive_macro_data(FRED_API_KEY, start_date='1990-01-01')

# # Separate columns by their data frequency (daily, monthly, quarterly, etc.)
# def infer_frequency(series):
#     # Drop NaNs and get sorted index
#     idx = series.dropna().index
#     if len(idx) < 2:
#         return 'unknown'
#     # Calculate median difference in days
#     freq_days = (idx[1:] - idx[:-1]).days
#     median_days = np.median(freq_days)
#     if median_days <= 2:
#         return 'daily'
#     elif 25 <= median_days <= 35:
#         return 'monthly'
#     elif 80 <= median_days <= 100:
#         return 'quarterly'
#     elif 350 <= median_days <= 370:
#         return 'yearly'
#     else:
#         return 'other'

# frequency_map = {}
# for col in macro_data.columns:
#     frequency_map[col] = infer_frequency(macro_data[col])

# daily_cols = [col for col, freq in frequency_map.items() if freq == 'daily']
# monthly_cols = [col for col, freq in frequency_map.items() if freq == 'monthly']
# quarterly_cols = [col for col, freq in frequency_map.items() if freq == 'quarterly']
# # yearly_cols = [col for col, freq in frequency_map.items() if freq == 'yearly']
# # other_cols = [col for col, freq in frequency_map.items() if freq == 'other']
# # unknown_cols = [col for col, freq in frequency_map.items() if freq == 'unknown']

# macro_data_daily = macro_data[daily_cols].dropna(how='all', axis=0)
# macro_data_monthly = macro_data[monthly_cols].dropna(how='all', axis=0)
# macro_data_quarterly = macro_data[quarterly_cols].dropna(how='all', axis=0)

# macro_data_daily.to_csv('data/processed/macro_data_daily.csv')
# macro_data_monthly.to_csv('data/processed/macro_data_monthly.csv')
# macro_data_quarterly.to_csv('data/processed/macro_data_quarterly.csv')

# Data Processing

In [9]:
fmp_idx = pd.read_csv('data/inputs/fmp_index_list.csv')
fmp_comm = pd.read_csv('data/inputs/fmp_commodity_list.csv')

symbol_name_dict = fmp_idx.set_index('symbol')['name'].to_dict()
symbol_name_dict.update(fmp_comm.set_index('symbol')['name'].to_dict())

### Read Macro Data from saved csv files

In [10]:
macro_data_daily = pd.read_csv('data/processed/macro_data_daily.csv', parse_dates=['date'], index_col='date')
macro_data_monthly = pd.read_csv('data/processed/macro_data_monthly.csv', parse_dates=['date'], index_col='date')
macro_data_quarterly = pd.read_csv('data/processed/macro_data_quarterly.csv', parse_dates=['date'], index_col='date')

macro_data_daily.columns = pd.MultiIndex.from_tuples([(col, macro_codes_dict[col], macro_units[col].split(',')[0]) for col in macro_data_daily.columns])
macro_data_monthly.columns = pd.MultiIndex.from_tuples([(col, macro_codes_dict[col], macro_units[col].split(',')[0]) for col in macro_data_monthly.columns])
macro_data_quarterly.columns = pd.MultiIndex.from_tuples([(col, macro_codes_dict[col], macro_units[col].split(',')[0]) for col in macro_data_quarterly.columns])

### Read Equity Index, Commodity, and FX data from saved csv files

In [11]:
# All commodities are in USD
commodity_data = pd.read_csv('data/processed/commodity_data.csv', index_col=0, header=[0,1])
commodity_data.index = pd.to_datetime(commodity_data.index)
commodity_data.tail()

ALIUSD                                               GCUSD  \
               open     high      low    close  volume     vwap    open   
date                                                                      
2025-09-22  2530.25  2544.75  2530.25  2530.25  2612.0  2533.88  3721.3   
2025-09-23  2536.00  2536.00  2522.00  2522.00    44.0  2529.00  3781.2   
2025-09-24  2533.50  2533.50  2533.50  2533.50  2113.0  2533.50  3796.9   
2025-09-25  2551.00  2551.00  2551.00  2551.00     1.0  2551.00  3768.3   
2025-09-26  2544.75  2544.75  2544.75  2544.75  1914.0  2544.75  3781.5   

                                    ...  BZUSD                         \
              high     low   close  ...   high    low  close   volume   
date                                ...                                 
2025-09-22  3783.3  3718.1  3775.1  ...  67.30  65.94  66.57  24400.0   
2025-09-23  3824.6  3772.4  3815.7  ...  68.08  66.12  67.63  27893.0   
2025-09-24  3812.6  3749.7  3768.1  ...  69.36  67.52  69.31  28459.0   
2025-09-25  3792.3  3751.9  3771.1  ...  69.67  68.44  69.42  28459.0   
2025-09-26  3814.4  3764.0  3809.0  ...  70.77  69.11  70.13  21388.0   

                       Nickel                                       
               vwap      open      high       low     close volume  
date                                                                
2025-09-22  66.6200  15261.75  15334.75  15183.88  15197.13    NaN  
2025-09-23  67.1050  15151.13  15352.13  15145.00  15335.38    NaN  
2025-09-24  68.5125  15268.25  15429.13  15250.38  15413.13    NaN  
2025-09-25  69.1550  15366.75  15491.38  15230.88  15252.88    NaN  
2025-09-26  69.9275  15194.75  15259.75  15131.88  15150.38    NaN  

[5 rows x 71 columns]

In [12]:
equity_index_data = pd.read_csv('data/processed/index_data.csv', index_col=0, header=[0,1])
equity_index_data.index = pd.to_datetime(equity_index_data.index)
equity_index_data.tail()

^GSPTSE                                                    \
               open     high      low    close       volume      vwap   
date                                                                    
2025-09-22  29791.0  29990.7  29747.9  29959.0  352957619.0  29872.15   
2025-09-23  30005.7  30066.6  29811.7  29815.6  296641424.0  29924.90   
2025-09-24  29865.0  30023.1  29755.2  29757.0  312079721.0  29850.07   
2025-09-25  29661.6  29768.3  29544.8  29732.0  293274800.0  29676.68   
2025-09-26  29778.4  29864.2  29739.2  29761.3  297599800.0  29785.77   

               ^TWII                                ...      ^GDAXI            \
                open      high       low     close  ...      volume      vwap   
date                                                ...                         
2025-09-22  25599.60  25887.69  25599.60  25880.60  ...  53681300.0  23527.06   
2025-09-23  26121.00  26307.30  26041.69  26247.37  ...  48153500.0  23625.96   
2025-09-24  26389.16  26394.03  26045.47  26196.73  ...  54163200.0  23626.22   
2025-09-25  26067.97  26247.85  26023.85  26023.85  ...  55308500.0  23542.11   
2025-09-26  25998.28  25998.28  25469.04  25580.32  ...  45110800.0  23668.99   

              ^FCHI                                                  \
               open     high      low    close      volume     vwap   
date                                                                  
2025-09-22  7847.76  7850.22  7801.17  7830.11  42599900.0  7832.31   
2025-09-23  7867.69  7919.07  7857.11  7872.02  48991400.0  7878.97   
2025-09-24  7871.22  7872.57  7810.05  7827.45  52936900.0  7845.32   
2025-09-25  7797.97  7819.70  7760.07  7795.42  51878400.0  7793.29   
2025-09-26  7822.17  7875.04  7821.25  7870.68  52920700.0  7847.29   

            MSCI_China         
                 close volume  
date                           
2025-09-22  190.769091    NaN  
2025-09-23  189.082017    NaN  
2025-09-24  192.635829    NaN  
2025-09-25  192.769367    NaN  
2025-09-26  189.959098    NaN  

[5 rows x 80 columns]

In [13]:
fx_data = pd.read_csv('data/processed/fx_data.csv', index_col=0)
fx_data.index = pd.to_datetime(fx_data.index)
fx_data.tail()

,CADUSD,TWDUSD,AUDUSD,JPYUSD,KRWUSD,EURUSD,GBPUSD,CHFUSD,INRUSD
date,,,,,,,,,
2025-09-22,0.7230,0.033099,0.65980,0.006767,0.000715,1.18025,1.35123,1.2607,0.011323
2025-09-23,0.7224,0.032954,0.65978,0.006773,0.000714,1.18148,1.35242,1.2629,0.011258
2025-09-24,0.7191,0.032913,0.65818,0.006719,0.000708,1.17380,1.34460,1.2575,0.011256
2025-09-25,0.7170,0.032698,0.65400,0.006675,0.000705,1.16660,1.33405,1.2492,0.011261
2025-09-26,0.7167,0.032815,0.65441,0.006687,0.000706,1.17002,1.33985,1.2525,0.011272


In [14]:
def update_level0_with_names(df, symbol_name_dict):
    # Only update if columns are MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        new_level0 = [f'{symbol_name_dict.get(sym, sym)} ({sym})' for sym in df.columns.get_level_values(0)]
        df.columns = pd.MultiIndex.from_arrays(
            [new_level0] + [df.columns.get_level_values(i) for i in range(1, df.columns.nlevels)],
            names=df.columns.names
        )
    return df

equity_index_data= update_level0_with_names(equity_index_data.copy(), symbol_name_dict)
commodity_data = update_level0_with_names(commodity_data.copy(), symbol_name_dict)

### Converting all Equity Indices to USD
Note: MSCI China Index is already in USD as it was downloaded directly from MSCI's website

In [15]:
# Convert all close prices to USD for indices not already in USD
usd_equity_index_data = equity_index_data.copy()

for idx, row in fmp_idx.iterrows():
    symbol = row['symbol']
    fx_symbol = row['fx_symbol']
    # Only convert if there is a corresponding fx_symbol (i.e., not already in USD)
    if fx_symbol and (symbol, 'close') in usd_equity_index_data.columns and fx_symbol in fx_data.columns:
        fx_series = fx_data[fx_symbol].reindex(usd_equity_index_data.index)
        usd_equity_index_data[(symbol, 'close')] = usd_equity_index_data[(symbol, 'close')] * fx_series

### Computing Derived Data Fields

In [16]:
equity_index_symbols = equity_index_data.droplevel(1, axis=1).columns.unique().to_list()
commodity_symbols = commodity_data.droplevel(1, axis=1).columns.unique().to_list()
all_symbols = equity_index_symbols + commodity_symbols

equity_index_symbols_names_dict = {row['symbol']: row['name'] for idx, row in fmp_idx.iterrows()}
equity_index_symbols_names_dict['MSCI_China'] = 'MSCI China Index'
commodity_symbols_names_dict = {row['symbol']: row['name'] for idx, row in fmp_comm.iterrows()}
commodity_symbols_names_dict['Nickel'] = 'Nickel Futures'
all_symbols_names_dict = {**equity_index_symbols_names_dict, **commodity_symbols_names_dict}

In [17]:
equities_derived = dd.TimeSeriesDerivedFields(price_data=usd_equity_index_data.xs(equity_index_symbols[0], level=0, axis=1)).compute_all_derived_fields()
equities_derived.columns = pd.MultiIndex.from_product([[equity_index_symbols[0]], equities_derived.columns])
for i in range(1, len(equity_index_symbols)):
    temp = dd.TimeSeriesDerivedFields(price_data=usd_equity_index_data.xs(equity_index_symbols[i], level=0, axis=1)).compute_all_derived_fields()
    temp.columns = pd.MultiIndex.from_product([[equity_index_symbols[i]], temp.columns])
    equities_derived = pd.concat([equities_derived, temp], axis=1)

In [18]:
commodities_derived = dd.TimeSeriesDerivedFields(price_data=commodity_data.xs(commodity_symbols[0], level=0, axis=1)).compute_all_derived_fields()
commodities_derived.columns = pd.MultiIndex.from_product([[commodity_symbols[0]], commodities_derived.columns])
for i in range(1, len(commodity_symbols)):
    temp = dd.TimeSeriesDerivedFields(price_data=commodity_data.xs(commodity_symbols[i], level=0, axis=1)).compute_all_derived_fields()
    temp.columns = pd.MultiIndex.from_product([[commodity_symbols[i]], temp.columns])
    commodities_derived = pd.concat([commodities_derived, temp], axis=1)